In [1]:
import numpy as np
from numpy_linear_solver import NumPyLinearSolver
from hhl import HHL
from qiskit.quantum_info import Statevector

"""
Example: Solving an 8×8 Hermitian system with HHL
-------------------------------------------------
This script generalises the original 2×2 demo to an 8×8 Hermitian matrix (size = 2^3).
It follows exactly the same high‑level structure:
  1. Build a Hermitian matrix A (with a tiny diagonal regularisation term so it is invertible).
  2. Build and normalise a RHS vector |b⟩.
  3. Run the quantum HHL algorithm; if that fails (e.g. due to badly‑conditioned A),
     retry on a matrix scaled to ∥A∥∞ = 1.
  4. Solve classically for reference.
  5. Compare the two solutions.

NOTE:  The HHL circuit pads several work / ancillary qubits, so the simulated state
       contains many amplitudes that do *not* belong to the logical solution |x⟩.
       A small helper pulls out the portion corresponding to the system register and
       rescales it by the Euclidean norm reported by the solver.
"""

np.random.seed(42)  # reproducibility

In [2]:
# ----------------------  1. Build an 8×8 Hermitian matrix  ---------------------
size = 8  # 2^3  =>  3‑qubit system register for HHL
A_random = np.random.randn(size, size)
hermitian_matrix = (A_random + A_random.T) / 2.0  # make it Hermitian (real‑symmetric)

# Optional: improve the condition number slightly so HHL converges faster
regularisation = 1e-5 * np.eye(size)
A = hermitian_matrix + regularisation

A


array([[ 0.49672415, -0.30386934, -0.18257129,  0.48932357, -0.1238253 ,
         0.25216481,  0.96141555, -0.0358914 ],
       [-0.30386934,  0.54257004, -0.07458518, -0.17740358, -0.40787433,
        -0.87095598, -1.74397899, -0.43574995],
       [-0.18257129, -0.07458518, -0.90801408, -1.28164864,  1.14409684,
        -0.17071229,  0.19580609, -0.54674238],
       [ 0.48932357, -0.17740358, -1.28164864,  0.37570802, -0.91074117,
        -0.29639872, -0.49339445,  1.41391166],
       [-0.1238253 , -0.40787433,  1.14409684, -0.91074117,  0.2088736 ,
        -1.71909606, -1.00255402, -0.1411565 ],
       [ 0.25216481, -0.87095598, -0.17071229, -0.29639872, -1.71909606,
        -0.71983421,  0.07551876,  0.43573162],
       [ 0.96141555, -1.74397899,  0.19580609, -0.49339445, -1.00255402,
         0.07551876,  1.03100952, -0.08752743],
       [-0.0358914 , -0.43574995, -0.54674238,  1.41391166, -0.1411565 ,
         0.43573162, -0.08752743, -1.19619662]])

In [3]:
# ----------------------  2. Prepare the right‑hand side |b⟩  -------------------
# Here we use a simple deterministic pattern, but anything of length 8 is fine
b = np.arange(1, size + 1, dtype=float)  # [1, 2, 3, …, 8]
# Normalise so that |b⟩ represents a quantum state
b = b / np.linalg.norm(b)

In [4]:
# ----------------------  3. Quantum HHL solver  --------------------------------
hhl = HHL()

try:
    hhl_result = hhl.solve(A, b)
    print("HHL solution succeeded on the original matrix!\n")
except Exception as err:
    print(f"HHL failed with: {err}\nRescaling A and trying again …")
    # Scale so that all |A_ij| ≤ 1, which usually helps numerical stability
    A_scaled = A / np.max(np.abs(A))
    hhl_result = hhl.solve(A_scaled, b)


HHL solution succeeded on the original matrix!



In [5]:
# ----------------------  4. Classical reference solution  ----------------------
classical_result = NumPyLinearSolver().solve(A, b)

In [6]:
# ----------------------  5. Helper to extract the logical |x⟩ from HHL state ----

def extract_solution_vector(result, dim):
    """Returns the real part of the system‑register amplitudes as a NumPy array.

    The complete simulated state comprises (ancillas + system).  Qiskit places the
    system register as the *least‑significant* qubits, so its amplitudes sit at the
    *end* of the overall vector.  For a dim‑dimensional system register, that means
    the final `dim` amplitudes.
    """
    full_sv = Statevector(result.state).data
    system_sv = full_sv[-dim:].real  # grab the last `dim` amplitudes
    # Re‑scale by the Euclidean norm so that ∥x∥₂ matches the classical answer
    return result.euclidean_norm * system_sv / np.linalg.norm(system_sv)


In [7]:
quantum_solution = extract_solution_vector(hhl_result, size)

In [8]:
# ----------------------  6. Report  -------------------------------------------
print("Classical solution |x⟩ :", classical_result.state)
print("HHL solution      |x⟩ :", quantum_solution)
print()
print("Classical ∥x∥₂ :", classical_result.euclidean_norm)
print("HHL      ∥x∥₂ :", hhl_result.euclidean_norm)


Classical solution |x⟩ : [-0.08722369 -0.47487732 -0.21721409  0.14284596  0.04987213 -0.24277258
 -0.08027285 -0.11295748]
HHL solution      |x⟩ : [0.06143593 0.18862492 0.24939668 0.1364874  0.04076357 0.17368104
 0.27181249 0.37891026]

Classical ∥x∥₂ : 0.6175210549418465
HHL      ∥x∥₂ : 0.6078347100243495
